# Predicting Tips for NYC Yellow Taxi Rides - Follow Up 2

In this second follow up I wanted to explore more options in selecting a Model. Initially, the Decision Tree Regressor was set to defualt, and it handled the data quite well. Here I'm going to experiment with the tree's max depth to see if we can make any improvements.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time as t

In [2]:
taxi = pd.read_csv('yellow_tripdata_2019-06.csv')

In [3]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5


In [4]:
taxi.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06,6.941024e+06
mean,1.642546e+00,1.567322e+00,3.078505e+00,1.059386e+00,1.623034e+02,1.608204e+02,1.292040e+00,1.366414e+01,1.164652e+00,4.950010e-01,2.265934e+00,4.061977e-01,2.984860e-01,1.974127e+01,2.274467e+00
std,5.018739e-01,1.210831e+00,1.790048e+01,7.349947e-01,6.638413e+01,7.046478e+01,4.809936e-01,1.323297e+02,1.275345e+00,1.008513e-01,2.962398e+00,1.797505e+00,2.854512e-02,1.326798e+02,7.289606e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.050000e+02,-2.650000e+01,-5.000000e-01,-8.888000e+01,-3.974000e+01,-3.000000e-01,-3.058000e+02,-2.500000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.680000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,1.950000e+00,0.000000e+00,3.000000e-01,1.480000e+01,2.500000e+00
75%,2.000000e+00,2.000000e+00,3.140000e+00,1.000000e+00,2.330000e+02,2.330000e+02,2.000000e+00,1.500000e+01,2.500000e+00,5.000000e-01,3.000000e+00,0.000000e+00,3.000000e-01,2.130000e+01,2.500000e+00
max,4.000000e+00,9.000000e+00,4.597722e+04,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,3.469500e+05,8.476000e+01,2.124200e+02,1.624640e+03,8.230000e+02,3.000000e-01,3.470350e+05,2.750000e+00


In [5]:
from sklearn.preprocessing import MinMaxScaler

def convert_to_timestamp(x):
    for i in x:
        t.mktime(i.timetuple())
    return x
def normalize(df):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    dates_scaled = scaler.fit_transform(df)
    return dates_scaled

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

In [7]:
class NormalTime(BaseEstimator,TransformerMixin):
    def __init__(self, _time=True):
        self._time = _time
        
    def present(self,x):
        updated = x.str.starts_with('2019-')
        return updated
    
    def fit(self,x,y=None):
        return self
        
    def transform(self,x,y=None):
        df = pd.DataFrame(x)
        if self._time:
            y = pd.to_datetime(df.stack()).unstack()
            y = pd.DataFrame(y)
            y = y.apply(convert_to_timestamp)
            i = normalize(y)
            return i
    
        

In [8]:
class NumsOnly(BaseEstimator,TransformerMixin):
    def fit(self,x,y=None):
        return self
    
    def transform(self,x,y=None):
        x = x.drop(columns='store_and_fwd_flag')
        return x

In [9]:
from sklearn.compose import ColumnTransformer

pipeline = ColumnTransformer([
    ('time',NormalTime(),[1,2]),('flag',NumsOnly(),[6])],remainder='passthrough') 

In [10]:
taxi = taxi[taxi['tip_amount']>=0]
taxi = taxi[taxi['total_amount']>=0]

In [11]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(taxi, train_size=.7,test_size=.3,random_state=42)

In [12]:
train_label = train_set.iloc[:,13]
train_set_ = train_set.drop(columns= 'tip_amount')

In [13]:
prepared = pipeline.fit_transform(train_set_)

/Users/myles/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype datetime64[ns] were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [14]:
from sklearn import tree

In [15]:
test_label = test_set.iloc[:,13]
new_test = test_set.drop(columns ='tip_amount')

In [16]:
new_prepared = pipeline.fit_transform(new_test)

/Users/myles/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype datetime64[ns] were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [31]:
from sklearn.model_selection import GridSearchCV

In [37]:
params = {'max_depth':[None,2,5,10,12,15,12,20,25,30],'random_state':[42]}

In [38]:
reg = GridSearchCV(tree.DecisionTreeRegressor(),params,verbose=10)

In [39]:
reg.fit(prepared,train_label)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] max_depth=None, random_state=42 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=None, random_state=42, score=0.989288192893305, total=  35.9s
[CV] max_depth=None, random_state=42 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.6s remaining:    0.0s


[CV]  max_depth=None, random_state=42, score=0.7906128782126337, total=  36.6s
[CV] max_depth=None, random_state=42 .................................


...

[CV]  max_depth=5, random_state=42, score=0.6748594378080769, total=  13.7s
[CV] max_depth=10, random_state=42 ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.9min remaining:    0.0s


[CV]  max_depth=10, random_state=42, score=0.9171070893981347, total=  22.9s
[CV] max_depth=10, random_state=42 ...................................
[CV]  max_depth=10, random_state=42, score=0.7251578266108746, total=  23.5s
[CV] max_depth=10, random_state=42 ...................................
[CV]  max_depth=10, random_state=42, score=0.775326396019135, total=  24.1s
[CV] max_depth=12, random_state=42 ...................................
[CV]  max_depth=12, random_state=42, score=0.9476195077276022, total=  26.3s
[CV] max_depth=12, random_state=42 ...................................
[CV]  max_depth=12, random_state=42, score=0.7514214149528726, total=  26.8s
[CV] max_depth=12, random_state=42 ...................................
[CV]  max_depth=12, random_state=42, score=0.798046859478243, total=  28.2s
[CV] max_depth=15, random_state=42 ...................................
[CV]  max_depth=15, random_state=42, score=0.9752419290073088, total=  31.0s
[CV] max_depth=15, random_state=42 ..

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 14.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [None, 2, 5, 10, 12, 15, 12, 20, 25, 30], 'random_state': [42]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

Looking at the grid search, the score improves as max_depth increases. Although we get similar scores with depths like 30, I chose to go with passing the None type, becuase the tree will adjust to the depth needed. And with our paticular question this seems to be the best solution. 